# EVALUATIONS ON THE ENSEMBLE ATTACK + FGSM
Models used: ResNet, SENet

In [1]:
import pandas as pd

from src.utils import *
import sys
import csv
import re
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_curve, auc

In [2]:
probs_res_ens1010_3_csv = '../eval/prob_ResNet_Ensemble_10_10_3dot0.csv'
probs_res_ens1010_3 = pd.read_csv(probs_res_ens1010_3_csv, header=0, engine='python')

probs_sen_ens1010_3_csv = '../eval/prob_SENet_Ensemble_10_10_3dot0.csv'
probs_sen_ens1010_3 = pd.read_csv(probs_sen_ens1010_3_csv, header=0, engine='python')

# AUDIO --------------------
#ResNet on ResNet audio
probs_res_FGSM_3s_ResNet_3_AUDIO_csv = '../eval/prob_ResNet_FGSM_ResNet_3s_3dot0_AUDIO.csv'
probs_res_FGSM_3s_ResNet_3_AUDIO = pd.read_csv(probs_res_FGSM_3s_ResNet_3_AUDIO_csv, header=0, engine='python')

#ResNet on SENet audio
probs_res_FGSM_3s_SENet_3_AUDIO_csv = '../eval/prob_ResNet_FGSM_SENet_3s_3dot0_AUDIO.csv'
probs_res_FGSM_3s_SEsNet_3_AUDIO = pd.read_csv(probs_res_FGSM_3s_SENet_3_AUDIO_csv, header=0, engine='python')

#SENet on SENet audio
probs_sen_FGSM_3s_SENet_3_AUDIO_csv = '../eval/prob_SENet_FGSM_SENet_3s_3dot0_AUDIO.csv'
probs_sen_FGSM_3s_SENet_3_AUDIO = pd.read_csv(probs_sen_FGSM_3s_SENet_3_AUDIO_csv, header=0, engine='python')

#SENet on ResNet audio
probs_sen_FGSM_3s_ResNet_3_AUDIO_csv = '../eval/prob_SENet_FGSM_ResNet_3s_3dot0_AUDIO.csv'
probs_sen_FGSM_3s_ResNet_3_AUDIO = pd.read_csv(probs_sen_FGSM_3s_ResNet_3_AUDIO_csv, header=0, engine='python')

# SPECTROGRAMS ------------------------
#ResNet on ResNet
probs_res_FGSM_3s_ResNet_3_SPEC_csv = '../eval/prob_ResNet_FGSM_ResNet_3s_3dot0_SPEC.csv'
probs_res_FGSM_3s_ResNet_3_SPEC = pd.read_csv(probs_res_FGSM_3s_ResNet_3_SPEC_csv, header=0, engine='python')

#ResNet on SENet spec
probs_res_FGSM_3s_SENet_3_SPEC_csv = '../eval/prob_ResNet_FGSM_SENet_3s_3dot0_SPEC.csv'
probs_res_FGSM_3s_SEsNet_3_SPEC = pd.read_csv(probs_res_FGSM_3s_SENet_3_SPEC_csv, header=0, engine='python')

#SENet on SENet
probs_sen_FGSM_3s_SENet_3_SPEC_csv = '../eval/prob_SENet_FGSM_SENet_3s_3dot0_SPEC.csv'
probs_sen_FGSM_3s_SENet_3_SPEC = pd.read_csv(probs_sen_FGSM_3s_SENet_3_SPEC_csv, header=0, engine='python')

#SENet on ResNet spec
probs_sen_FGSM_3s_ResNet_3_SPEC_csv = '../eval/prob_SENet_FGSM_ResNet_3s_3dot0_SPEC.csv'
probs_sen_FGSM_3s_ResNet_3_SPEC = pd.read_csv(probs_sen_FGSM_3s_ResNet_3_SPEC_csv, header=0, engine='python')

In [3]:
# they all work on the same dataset so they should all have the same length
print(f'length of probs_res_ens1010_3: {len(probs_res_ens1010_3)}\n'
      f'length of probs_sen_ens1010_3: {len(probs_sen_ens1010_3)}\n'
      f'length of probs_res_FGSM_3s_ResNet_3_AUDIO: {len(probs_res_FGSM_3s_ResNet_3_AUDIO)}\n'
      f'length of probs_sen_FGSM_3s_SENet_3_AUDIO: {len(probs_sen_FGSM_3s_SENet_3_AUDIO)}\n'
      f'length of probs_res_FGSM_3s_ResNet_3_SPEC: {len(probs_res_FGSM_3s_ResNet_3_SPEC)}\n'
      f'length of probs_sen_FGSM_3s_SENet_3_SPEC: {len(probs_sen_FGSM_3s_SENet_3_SPEC)}')

length of probs_res_ens1010_3: 31488
length of probs_sen_ens1010_3: 31488
length of probs_res_FGSM_3s_ResNet_3_AUDIO: 31488
length of probs_sen_FGSM_3s_SENet_3_AUDIO: 31488
length of probs_res_FGSM_3s_ResNet_3_SPEC: 31488
length of probs_sen_FGSM_3s_SENet_3_SPEC: 31488


In [4]:
# GT labels
eval_csv = '../data/df_eval_19_3s.csv'
df_eval = pd.read_csv(eval_csv)
GT_labels = df_eval.iloc[:,-1].tolist()
print(f'Length of GT labels list: {len(GT_labels)}')

Length of GT labels list: 31489


***
## Extract predicted labels in the same order as in df_eval_19_3s

In [5]:
# Function to extract the file number from a given filename
def extract_file_number(filename):
    match = re.search(r"LA_E_(\d+)", filename)
    if match:
        return match.group(1)
    return None

def GT_and_predicted_labels(file2_path):
    # Read file 1 and store the file number and label in a dictionary
    file1_dict = {}
    with open('../data/df_eval_19_3s.csv', 'r') as file1:
        reader = csv.reader(file1)
        next(reader)  # Skip header
        for row in reader:
            # Extract file number from the path (assuming path contains 'LA_E_xxxxxxx.flac')
            match = re.search(r"LA_E_(\d+)", row[1])
            if match:
                file_number = match.group(1)
                label = int(row[2])  # Convert label to integer
                file1_dict[file_number] = label  # Store file number and corresponding label
    
    # Initialize arrays for predicted classes and true labels
    predicted_classes = []
    true_labels = []
    
    # Read file 2 and process the data
    with open(file2_path, 'r') as file2:
        reader = csv.reader(file2)
        next(reader)  # Skip header
        for row in reader:
            filename = row[0]  # Filename is in the first column
            pred_class_0 = float(row[1])  # Prediction for class 0
            pred_class_1 = float(row[2])  # Prediction for class 1
        
            # Determine predicted class: 0 if Pred.class 0 > Pred.class 1, else 1
            if pred_class_0 > pred_class_1:
                predicted_classes.append(0)
            else:
                predicted_classes.append(1)

            # Extract the file number
            file_number = extract_file_number(filename)

            # Get the true label from file1_dict if the file number exists
            if file_number in file1_dict:
                true_labels.append(file1_dict[file_number])
            else:
                true_labels.append(None)  # If no matching file number is found in file 1
    return predicted_classes, true_labels



***
## Labels and GT

In [6]:
pred_labels_res_ens1010_3, GT_res_ens1010_3 = GT_and_predicted_labels(file2_path=probs_res_ens1010_3_csv)
pred_labels_sen_ens1010_3, GT_sen_ens1010_3 = GT_and_predicted_labels(file2_path=probs_sen_ens1010_3_csv)

pred_labels_res_FGSM_ResNet_3s_3_AUDIO, GT_res_FGSM_ResNet_3s_3_AUDIO = GT_and_predicted_labels(file2_path=probs_res_FGSM_3s_ResNet_3_AUDIO_csv)
pred_labels_sen_FGSM_SENet_3s_3_AUDIO, GT_sen_FGSM_SENet_3s_3_AUDIO = GT_and_predicted_labels(file2_path=probs_sen_FGSM_3s_SENet_3_AUDIO_csv)

pred_labels_res_FGSM_SENet_3s_3_AUDIO, GT_res_FGSM_SENet_3s_3_AUDIO = GT_and_predicted_labels(file2_path=probs_res_FGSM_3s_SENet_3_AUDIO_csv)
pred_labels_sen_FGSM_ResNet_3s_3_AUDIO, GT_sen_FGSM_ResNet_3s_3_AUDIO = GT_and_predicted_labels(file2_path=probs_sen_FGSM_3s_ResNet_3_AUDIO_csv)

pred_labels_res_FGSM_ResNet_3s_3_SPEC, GT_res_FGSM_ResNet_3s_3_SPEC = GT_and_predicted_labels(file2_path=probs_res_FGSM_3s_ResNet_3_SPEC_csv)
pred_labels_sen_FGSM_SENet_3s_3_SPEC, GT_sen_FGSM_SENet_3s_3_SPEC = GT_and_predicted_labels(file2_path=probs_sen_FGSM_3s_SENet_3_SPEC_csv)

pred_labels_res_FGSM_SENet_3s_3_SPEC, GT_res_FGSM_SENet_3s_3_SPEC = GT_and_predicted_labels(file2_path=probs_res_FGSM_3s_SENet_3_SPEC_csv)
pred_labels_sen_FGSM_ResNet_3s_3_SPEC, GT_sen_FGSM_ResNet_3s_3_SPEC = GT_and_predicted_labels(file2_path=probs_sen_FGSM_3s_ResNet_3_SPEC_csv)

***
## Unbalanced accuracy

In [7]:
UA_res_ens1010_3 = accuracy_score(y_true=GT_res_ens1010_3, y_pred=pred_labels_res_ens1010_3)
UA_sen_ens1010_3 = accuracy_score(y_true=GT_sen_ens1010_3, y_pred=pred_labels_sen_ens1010_3)

print(
    f'AUDIO -- The unbalanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is {UA_res_ens1010_3*100:.2f}%\n'
    f'AUDIO -- The unbalanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is {UA_sen_ens1010_3*100:.2f}%')

AUDIO -- The unbalanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is 32.47%
AUDIO -- The unbalanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is 54.64%


In [8]:
UA_res_FGSM_ResNet_3s_3_AUDIO = accuracy_score(y_true=GT_res_FGSM_ResNet_3s_3_AUDIO, y_pred=pred_labels_res_FGSM_ResNet_3s_3_AUDIO)
UA_sen_FGSM_SENet_3s_3_AUDIO = accuracy_score(y_true=GT_sen_FGSM_SENet_3s_3_AUDIO, y_pred=pred_labels_sen_FGSM_SENet_3s_3_AUDIO)

UA_res_FGSM_SENet_3s_3_AUDIO = accuracy_score(y_true=GT_res_FGSM_SENet_3s_3_AUDIO, y_pred=pred_labels_res_FGSM_SENet_3s_3_AUDIO)
UA_sen_FGSM_ResNet_3s_3_AUDIO = accuracy_score(y_true=GT_sen_FGSM_ResNet_3s_3_AUDIO, y_pred=pred_labels_sen_FGSM_ResNet_3s_3_AUDIO)

print(f'AUDIO -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is {UA_res_FGSM_ResNet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The unbalanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is {UA_sen_FGSM_SENet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The unbalanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is {UA_res_FGSM_SENet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The unbalanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is {UA_sen_FGSM_ResNet_3s_3_AUDIO*100:.2f}%')

AUDIO -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is 12.20%
AUDIO -- The unbalanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is 49.94%
AUDIO -- The unbalanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is 84.87%
AUDIO -- The unbalanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is 69.59%


In [9]:
UA_res_FGSM_ResNet_3s_3_SPEC = accuracy_score(y_true=GT_res_FGSM_ResNet_3s_3_SPEC, y_pred=pred_labels_res_FGSM_ResNet_3s_3_SPEC)
UA_sen_FGSM_SENet_3s_3_SPEC = accuracy_score(y_true=GT_sen_FGSM_SENet_3s_3_SPEC, y_pred=pred_labels_sen_FGSM_SENet_3s_3_SPEC)

UA_res_FGSM_SENet_3s_3_SPEC = accuracy_score(y_true=GT_res_FGSM_SENet_3s_3_SPEC, y_pred=pred_labels_res_FGSM_SENet_3s_3_SPEC)
UA_sen_FGSM_ResNet_3s_3_SPEC = accuracy_score(y_true=GT_sen_FGSM_ResNet_3s_3_SPEC, y_pred=pred_labels_sen_FGSM_ResNet_3s_3_SPEC)

print(f'SPEC -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is {UA_res_FGSM_ResNet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is {UA_sen_FGSM_SENet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The unbalanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is {UA_res_FGSM_SENet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The unbalanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is {UA_sen_FGSM_ResNet_3s_3_SPEC*100:.2f}%')

SPEC -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is 0.00%
SPEC -- The unbalanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is 11.28%
SPEC -- The unbalanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is 84.48%
SPEC -- The unbalanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is 75.87%


***
## Balanced accuracy

In [10]:
BA_res_ens1010_3 = balanced_accuracy_score(y_true=GT_res_ens1010_3, y_pred=pred_labels_res_ens1010_3)
BA_sen_ens1010_3 = balanced_accuracy_score(y_true=GT_sen_ens1010_3, y_pred=pred_labels_sen_ens1010_3)

print(
    f'AUDIO -- The balanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is {BA_res_ens1010_3*100:.2f}%\n'
    f'AUDIO -- The balanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is {BA_sen_ens1010_3*100:.2f}%')

AUDIO -- The balanced accuracy for ResNet on the ensemble attack 10 10 epsilon=3.0 is 22.28%
AUDIO -- The balanced accuracy for SENet on the ensemble attack 10 10 epsilon=3.0 is 64.09%


In [11]:
BA_res_FGSM_ResNet_3s_3_AUDIO = balanced_accuracy_score(y_true=GT_res_FGSM_ResNet_3s_3_AUDIO, y_pred=pred_labels_res_FGSM_ResNet_3s_3_AUDIO)
BA_sen_FGSM_SENet_3s_3_AUDIO = balanced_accuracy_score(y_true=GT_sen_FGSM_SENet_3s_3_AUDIO, y_pred=pred_labels_sen_FGSM_SENet_3s_3_AUDIO)

BA_res_FGSM_SENet_3s_3_AUDIO = balanced_accuracy_score(y_true=GT_res_FGSM_SENet_3s_3_AUDIO, y_pred=pred_labels_res_FGSM_SENet_3s_3_AUDIO)
BA_sen_FGSM_ResNet_3s_3_AUDIO = balanced_accuracy_score(y_true=GT_sen_FGSM_ResNet_3s_3_AUDIO, y_pred=pred_labels_sen_FGSM_ResNet_3s_3_AUDIO)

print(f'AUDIO -- The balanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is {BA_res_FGSM_ResNet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The balanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is {BA_sen_FGSM_SENet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The balanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is {BA_res_FGSM_SENet_3s_3_AUDIO*100:.2f}%\n'
      f'AUDIO -- The balanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is {BA_sen_FGSM_ResNet_3s_3_AUDIO*100:.2f}%')

AUDIO -- The balanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is 7.17%
AUDIO -- The balanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is 60.23%
AUDIO -- The balanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is 88.03%
AUDIO -- The balanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is 76.72%


In [12]:
BA_res_FGSM_ResNet_3s_3_SPEC = balanced_accuracy_score(y_true=GT_res_FGSM_ResNet_3s_3_SPEC, y_pred=pred_labels_res_FGSM_ResNet_3s_3_SPEC)
BA_sen_FGSM_SENet_3s_3_SPEC = balanced_accuracy_score(y_true=GT_sen_FGSM_SENet_3s_3_SPEC, y_pred=pred_labels_sen_FGSM_SENet_3s_3_SPEC)

BA_res_FGSM_SENet_3s_3_SPEC = balanced_accuracy_score(y_true=GT_res_FGSM_SENet_3s_3_SPEC, y_pred=pred_labels_res_FGSM_SENet_3s_3_SPEC)
BA_sen_FGSM_ResNet_3s_3_SPEC = balanced_accuracy_score(y_true=GT_sen_FGSM_ResNet_3s_3_SPEC, y_pred=pred_labels_sen_FGSM_ResNet_3s_3_SPEC)

print(f'SPEC -- The balanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is {BA_res_FGSM_ResNet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The balanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is {BA_sen_FGSM_SENet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The balanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is {BA_res_FGSM_SENet_3s_3_SPEC*100:.2f}%\n'
      f'SPEC -- The balanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is {BA_sen_FGSM_ResNet_3s_3_SPEC*100:.2f}%')

SPEC -- The balanced accuracy for ResNet on FGSM ResNet 3s and epsilon=3.0 is 0.00%
SPEC -- The balanced accuracy for SENet on FGSM SENet 3s and epsilon=3.0 is 6.54%
SPEC -- The balanced accuracy for ResNet on FGSM SENet 3s and epsilon=3.0 is 88.01%
SPEC -- The balanced accuracy for SENet on FGSM ResNet 3s and epsilon=3.0 is 78.62%
